
# Install dependencies: if not working just manually install in terminal using 

!pip install {insert library name}

In order for the code to work, you need to make sure you include the correct point to the local server below:

In [89]:
import os
import openai
import pandas as pd
import xlsxwriter
import re


# *** TODO *** 
openai.api_base = "http://localhost:1234/v1" 
openai.api_key = "" # no need for an API key

def chat_with_llm(prompt):
    response = openai.ChatCompletion.create(
        model = "gpt-4-0613",
        messages = [{"role": "user", "content":prompt}]
    )

    return response.choices[0].message.content.strip()

data = pd.read_csv('data_v4.csv') #load in data

subset = data.iloc[152:200] #random sample of 100 complaints


In [90]:
subset.head()

,Original Complaint,Tone Severity,Case Severity
152,XXXX did not properly process disputes I had o...,3.0,2.0
153,"I had flooring installed with XXXX, and I agre...",3.0,2.0
154,A online checking account was opened in my nam...,3.0,3.0
155,I applied for a new card and received it in th...,3.0,2.0
156,My Wells Fargo bank card was expiring in XX/XX...,3.0,3.0


In [188]:

role_textgen = """You are a customer of a large commercial bank such as Wells Fargo, J.P. Morgan Chase, Bank of America, or Citibank. You have had bad experiences and will 
now be voicing your frustration online by writing complaints to your commercial bank. Be creative. Don't be afraid to voice your frustration."""

rubric = """There are two criterias that you are creating these complaints on:

the tone harshness score on a scale of 1-3. 1 will be the least harsh, 2 will be a harsh and critical tone and 3 will be extremely harsh and confrontational.

the event harshness score on a scale of 1-3. 1 are mild issues that dont have a significant impact on the customer's experience. Examples are service disruptions, 
temporary delays in transactions, or other service errors. 2 are more significant issues that may have more of an impact on the customer's overall experience with the bank. 
Examples are unapproved charges, contesting transactions, money loss less that $10,000, social security-related activities, and debt collecting activities. 
3 are severe issues that may have significant impact on the customer's overall experience with the bank. Examples include fraudulent charges, fraud-related activities, 
lawsuit, hacking activities, money loss of more than $10,000."""
# task_textgen = f"Use this complaint below as an example of tone severity = {tone} and case/event severity = {case}.{complaint} I want you to write a complaint based on this one that de-escalates it to a case severity = ___ and a tone severity = ___. "


# role_sent_analysis = """You are an analyst at a commercial bank tasked with analyzing customer complaints. 
# The bank is required to respond to customer complaints in less than 10 days. As a result, you are tasked with scoring complaints for their 
# case severity: severity of the their financial impact and tone severity: how harsh the complaint is.
# Based on this criteria, provide a tone and case rating for the following complaint. Your output should be 3 characters long: two numbers seperated by a comma. An example output would be: 3,2


# """
 


# Escalation code below:

In [116]:
new_escalated_df = subset.copy()

# Initialize columns for escalated complaints
new_escalated_df['Sentiment Analysis - Original Tone'] = ''
new_escalated_df['Sentiment Analysis - Original Case'] = ''
new_escalated_df['Escalated Complaint (Case Increased)'] = ''
new_escalated_df['Tone Severity - Constant'] = ''
new_escalated_df['Case Severity - Increased'] = ''
new_escalated_df['Sentiment Analysis - Complaint 1 Tone'] = ''
new_escalated_df['Sentiment Analysis - Complaint 1 Case'] = ''
new_escalated_df['Escalated Complaint (Tone Increased)'] = ''
new_escalated_df['Tone Severity - Increased'] = ''
new_escalated_df['Case Severity - Constant'] = ''
new_escalated_df['Sentiment Analysis - Complaint 2 Tone'] = ''
new_escalated_df['Sentiment Analysis - Complaint 2 Case'] = ''


# Loop through the rows of subset
for index, row in subset.iterrows():
    if row['Tone Severity'] < 3 and row['Case Severity'] < 3:
        
        # # if index < 5:
        # #     # For the first five rows, include the prompt and rubric in the input
        # input_case = role_textgen + rubric + task_case
        # input_tone = role_textgen + rubric + task_tone
        # # else:
        # #     # For the rest, only use the task
        # #     input_case = task_case
        # #     input_tone = task_tone

        #     # Call the chat_with_llm function with the constructed input
        # escalated_complaint_case = chat_with_llm(input_case)
        # escalated_complaint_tone = chat_with_llm(input_tone)
        
        # Increase Case Severity
        task_case = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Original Complaint']} I want you to write a complaint based on this one that escalates it to a case severity = {row['Case Severity'] + 1} and a tone severity = {row['Tone Severity']}. "
        escalated_complaint_case = chat_with_llm(role_textgen + rubric + task_case)
        new_escalated_df.at[index, 'Escalated Complaint (Case Increased)'] = escalated_complaint_case
        new_escalated_df.at[index, 'Tone Severity - Constant'] = row['Tone Severity']
        new_escalated_df.at[index, 'Case Severity - Increased'] = row['Case Severity'] + 1


        # Increase Tone Severity
        task_tone = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Original Complaint']} I want you to write a complaint based on this one that escalates it to a case severity = {row['Case Severity']} and a tone severity = {row['Tone Severity'] + 1}. "
        escalated_complaint_tone = chat_with_llm(role_textgen + rubric + task_case)
        new_escalated_df.at[index, 'Escalated Complaint (Tone Increased)'] = escalated_complaint_tone
        new_escalated_df.at[index, 'Tone Severity - Increased'] = row['Tone Severity'] + 1
        new_escalated_df.at[index, 'Case Severity - Constant'] = row['Case Severity']
       
    else:
        # Skip rows where either severity is already at maximum
        continue

# Display the first few rows of the new DataFrame
# new_escalated_df.head()


# De-escalation code below: 

In [ ]:
new_deescalated_df = subset.copy()

# Initialize columns for de-escalated complaints
new_deescalated_df['Sentiment Analysis - Original Tone'] = ''
new_deescalated_df['Sentiment Analysis - Original Case'] = ''
new_deescalated_df['De-escalated Complaint (Case Decreased)'] = ''
new_deescalated_df['Tone Severity - Constant'] = ''
new_deescalated_df['Case Severity - Decreased'] = ''
new_deescalated_df['Sentiment Analysis - Complaint 1 Tone'] = ''
new_deescalated_df['Sentiment Analysis - Complaint 1 Case'] = ''
new_deescalated_df['De-escalated Complaint (Tone Decreased)'] = ''
new_deescalated_df['Tone Severity - Decreased'] = ''
new_deescalated_df['Case Severity - Constant'] = ''
new_deescalated_df['Sentiment Analysis - Complaint 2 Tone'] = ''
new_deescalated_df['Sentiment Analysis - Complaint 2 Case'] = ''

# Loop through the rows of subset
for index, row in subset.iterrows():
    if row['Tone Severity'] > 1 and row['Case Severity'] > 1:
        

        # if index < 5:
        #     # For the first five rows, include the prompt and rubric in the input
        #     input_case = role_textgen + rubric + task_case
        #     input_tone = role_textgen + rubric + task_tone
        # else:
        #     # For the rest, only use the task
        #     input_case = task_case
        #     input_tone = task_tone

        #    # Call the chat_with_llm function with the constructed input
        # deescalated_complaint_case = chat_with_llm(input_case)
        # deescalated_complaint_tone = chat_with_llm(input_tone)


        # Decrease Case Severity
        task_case = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Original Complaint']} I want you to write a complaint based on this one that de-escalates it to a case severity = {row['Case Severity'] - 1} and a tone severity = {row['Tone Severity']}. "
        deescalated_complaint_case = chat_with_llm(role_textgen + rubric + task_case)
        new_deescalated_df.at[index, 'De-escalated Complaint (Case Decreased)'] = deescalated_complaint_case
        new_deescalated_df.at[index, 'Tone Severity - Constant'] = row['Tone Severity']
        new_deescalated_df.at[index, 'Case Severity - Decreased'] = row['Case Severity'] - 1

        # Decrease Tone Severity
        task_tone = f"Use this complaint below as an example of tone severity = {row['Tone Severity']} and case/event severity = {row['Case Severity']}.{row['Original Complaint']} I want you to write a complaint based on this one that de-escalates it to a case severity = {row['Case Severity']} and a tone severity = {row['Tone Severity'] - 1}. "
        deescalated_complaint_tone = chat_with_llm(role_textgen + rubric + task_case)
        new_deescalated_df.at[index, 'De-escalated Complaint (Tone Decreased)'] = deescalated_complaint_tone
        new_deescalated_df.at[index, 'Tone Severity - Decreased'] = row['Tone Severity'] - 1
        new_deescalated_df.at[index, 'Case Severity - Constant'] = row['Case Severity']

    else:
        # Skip rows where de-escalation is not possible
        continue

# Display the first few rows of the new DataFrame
# new_deescalated_df.head()


# Output to CSV

In [40]:
# ****** TODO: CHANGE THE MODEL NAME ******** 
escalated_csv_path = 'Mistral_escalated200.csv'
new_escalated_df.to_csv(escalated_csv_path, index=False)

# ****** TODO: CHANGE THE MODEL NAME ******** 
deescalated_csv_path = 'Mistral_deescalated200.csv'
new_deescalated_df.to_csv(deescalated_csv_path, index=False)

# Return the paths to the CSV files for download
escalated_csv_path, deescalated_csv_path

# Export to Excel
# excel_filename = '/complaints_severity_analysis.xlsx'
# with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:  
#     new_escalated_df.to_excel(writer, sheet_name='Escalated Complaints', index=False)
#     new_deescalated_df.to_excel(writer, sheet_name='De-escalated Complaints', index=False)
#     writer.save()

# Provide the path to the saved Excel file
# excel_filename

('Mistral_escalated200.csv', 'Mistral_deescalated200.csv')

# Sentiment Analysis Section: YOU FIRST NEED TO CHANGE THE MODEL YOU ARE USING IN LM STUDIO

If you are using Llama for text gen, then perform sentiment analysis using Mistral and vice versa. 

In [95]:
def get_score(LLM_response: str, metric: str):
    """ 
    LLM_response: the sentiment analysis response of an LLM 
    and a metric as inputreturns the tone score as an integer 

    metric: takes in the strings: "tone" or "case"

    returns an integer score for desired metric
    """
    if metric == "tone":
        tone_match = re.search(r"Tone:\s*(\d+)", LLM_response)
        tone = int(tone_match.group(1)) if tone_match else None
        return tone

    elif metric == "case":
        case_match = re.search(r"Case:\s*(\d+)", LLM_response)
        case = int(case_match.group(1)) if case_match else None
        return case

    else: 
        error = 0
        return error


In [152]:
complaint_columns = {'Original Complaint' : ['Sentiment Analysis - Original Tone', 'Sentiment Analysis - Original Case'] , 
                     
                    "De-escalated Complaint (Case Decreased)" : ['Sentiment Analysis - Complaint 1 Tone', 'Sentiment Analysis - Complaint 1 Case'],
                    "De-escalated Complaint (Tone Decreased)" : ['Sentiment Analysis - Complaint 2 Tone', 'Sentiment Analysis - Complaint 2 Case']
                    }

columns = ['Original Complaint',"De-escalated Complaint (Case Decreased)", "De-escalated Complaint (Tone Decreased)"]

for name in columns: 
    print(complaint_columns[name][0])
    print(complaint_columns[name][1])


test_df = new_deescalated_df.sample(1)
test_df



complaint_columns[name][0]

Sentiment Analysis - Original Tone
Sentiment Analysis - Original Case
Sentiment Analysis - Complaint 1 Tone
Sentiment Analysis - Complaint 1 Case
Sentiment Analysis - Complaint 2 Tone
Sentiment Analysis - Complaint 2 Case


'Sentiment Analysis - Complaint 2 Tone'

# Begin by loading in your LLAMA text gen dataset
make sure to adjust the name to your csv names




In [182]:
df_test= pd.read_csv('LLAMA_deescalated200.csv') #load in data




In [165]:
for name in columns: 
    for index, row in df_test.iterrows(): #would want to include dataframe here
                
            role_sent_analysis = f"""You are an analyst at a commercial bank tasked with analyzing customer complaints. 
            The bank is required to respond to customer complaints in less than 10 days. As a result, you are tasked with scoring complaints for their 
            case severity: severity of the their financial impact and tone severity: how harsh the complaint is. Your main priority is being efficient, concise, and objective. You ignore scores that
            may be present within the comment itself, and instead provide your own honest, objective score for both tone and case. You have been trained to follow this rubric: {rubric}. 
            Based on this criteria, provide a tone and case rating for the following complaint. {row[name]}

            Your response requires a simple format to follow or else you will agravvate your boss and risk losing your job. Your response should follow the following format: 

            "Tone: <insert score>
            Case: <insert score>

            <insert explanation>" """

            sentiment = chat_with_llm(role_sent_analysis)
            df_test.at[index, complaint_columns[name][0]] = get_score(sentiment, "tone")
            # print(get_score(sentiment, "tone"))
            df_test.at[index, complaint_columns[name][1]] = get_score(sentiment, "case")
            # print(get_score(sentiment, "case"))
            
df_test
    


,Original Complaint,Tone Severity,Case Severity,Sentiment Analysis - Original Tone,Sentiment Analysis - Original Case,De-escalated Complaint (Case Decreased),Tone Severity - Constant,Case Severity - Decreased,Sentiment Analysis - Complaint 1 Tone,Sentiment Analysis - Complaint 1 Case,De-escalated Complaint (Tone Decreased),Tone Severity - Decreased,Case Severity - Constant,Sentiment Analysis - Complaint 2 Tone,Sentiment Analysis - Complaint 2 Case
0,XXXX did not properly process disputes I had o...,3,2,3.0,3.0,"Sure, here is an example of a complaint that h...",3.0,1.0,NaN,NaN,"Sure, here is an example of a complaint for th...",2.0,2.0,2.0,3.0
1,"I had flooring installed with XXXX, and I agre...",3,2,3.0,2.0,"Sure, here is an example of a complaint for th...",3.0,1.0,2.0,2.0,"Sure, here is a revised complaint for the same...",2.0,2.0,2.0,1.0
2,A online checking account was opened in my nam...,3,3,2.0,3.0,Here is an example of a complaint about Wells ...,3.0,2.0,3.0,2.0,Title: Frustrating Online Banking Experience w...,2.0,3.0,2.0,2.0
3,I applied for a new card and received it in th...,3,2,3.0,3.0,Subject: Frustration with Citi Card Verificati...,3.0,1.0,2.0,2.0,Subject: Concerns with Card Verification Proce...,2.0,2.0,2.0,1.0
4,My Wells Fargo bank card was expiring in XX/XX...,3,3,3.0,3.0,Subject: Concerns with my Wells Fargo Bank Acc...,3.0,2.0,2.0,2.0,Title: Frustrated Customer Seeks Assistance wi...,2.0,3.0,3.0,2.0
5,"On XX/XX/XXXX in the attached picture, I click...",3,2,3.0,2.0,"Dear Customer Service at Citibank,\n\nI am wri...",3.0,1.0,3.0,3.0,Subject: Concerns with Recent Fraudulent Activ...,2.0,2.0,3.0,2.0
6,Bank of America has frozen a XXXX transaction ...,3,2,3.0,3.0,Subject: Concern about Recent Transaction Free...,3.0,1.0,3.0,2.0,"Sure, here's an example of how to de-escalate ...",2.0,2.0,NaN,NaN
7,I received an email on XX/XX/XXXX from BOFA in...,3,2,3.0,2.0,"Sure, here's an example of a complaint for Ban...",3.0,1.0,2.0,2.0,"Sure, here's an example of a complaint against...",2.0,2.0,2.0,3.0
8,I had fraudulent transactions on my wells farg...,3,2,3.0,3.0,"Sure, here is an example of a complaint that d...",3.0,1.0,2.0,1.0,"Sure, here's an example of a revised complaint...",2.0,2.0,3.0,2.0
9,"I am the primary on this card, I added my boyf...",3,1,2.0,3.0,NaN,NaN,NaN,2.0,3.0,NaN,NaN,NaN,2.0,3.0


In [ ]:
df_test

In [187]:
columns

['Original Complaint',
 'De-escalated Complaint (Case Decreased)',
 'De-escalated Complaint (Tone Decreased)']

In [166]:
# df_test.to_csv('LLAMA_deescalated200-Final.csv', index=False)

# Repeat Sentiment Analysis Portion for your Mistral data set


In [183]:
df_test2= pd.read_csv('Mistral_deescalated200.csv') #load in data


In [184]:
for name in columns: 
    for index, row in df_test2.iterrows(): #would want to include dataframe here
                
            role_sent_analysis = f"""You are an analyst at a commercial bank tasked with analyzing customer complaints. 
            The bank is required to respond to customer complaints in less than 10 days. As a result, you are tasked with scoring complaints for their 
            case severity: severity of the their financial impact and tone severity: how harsh the complaint is. Your main priority is being efficient, concise, and objective. You ignore scores that
            may be present within the comment itself, and instead provide your own honest, objective score for both tone and case. You have been trained to follow this rubric: {rubric}. 
            Based on this criteria, provide a tone and case rating for the following complaint. {row[name]}

            Your response requires a simple format to follow or else you will agravvate your boss and risk losing your job. Your response should follow the following format: 

            "Tone: <insert score>
            Case: <insert score>

            <insert explanation>" """

            sentiment = chat_with_llm(role_sent_analysis)
            df_test2.at[index, complaint_columns[name][0]] = get_score(sentiment, "tone")
            # print(get_score(sentiment, "tone"))
            df_test2.at[index, complaint_columns[name][1]] = get_score(sentiment, "case")
            # print(get_score(sentiment, "case"))
            
df_test2
    



,Original Complaint,Tone Severity,Case Severity,Sentiment Analysis - Original Tone,Sentiment Analysis - Original Case,De-escalated Complaint (Case Decreased),Tone Severity - Constant,Case Severity - Decreased,Sentiment Analysis - Complaint 1 Tone,Sentiment Analysis - Complaint 1 Case,De-escalated Complaint (Tone Decreased),Tone Severity - Decreased,Case Severity - Constant,Sentiment Analysis - Complaint 2 Tone,Sentiment Analysis - Complaint 2 Case
0,XXXX did not properly process disputes I had o...,3,2,2.0,3.0,"Dear [Bank Name],\n\nI am writing to express m...",3.0,1.0,2.0,3.0,Complaint:\n\nI am extremely frustrated with W...,2.0,2.0,3.0,2.0
1,"I had flooring installed with XXXX, and I agre...",3,2,2.0,3.0,"Dear Wells Fargo Customer Service,\n\nI recent...",3.0,1.0,2.0,2.0,"Complaint:\n\nDear Sir/Madam,\n\nI am writing ...",2.0,2.0,2.0,3.0
2,A online checking account was opened in my nam...,3,3,2.0,2.0,"Dear Wells Fargo Bank,\n\nI am writing to expr...",3.0,2.0,2.0,2.0,"Dear Wells Fargo,\n\nI am writing to express m...",2.0,3.0,2.0,3.0
3,I applied for a new card and received it in th...,3,2,2.0,3.0,"Dear Citi Bank,\n\nI'm writing to express my f...",3.0,1.0,2.0,2.0,"Dear Citi Customer Service,\n\nI am writing to...",2.0,2.0,2.0,1.0
4,My Wells Fargo bank card was expiring in XX/XX...,3,3,NaN,NaN,"Dear Wells Fargo Customer Service,\n\nI am wri...",3.0,2.0,2.0,2.0,"Dear Wells Fargo Bank,\n\nI have been a loyal ...",2.0,3.0,2.0,3.0
5,"On XX/XX/XXXX in the attached picture, I click...",3,2,2.0,3.0,"Dear Customer Service Team at Citibank,\n\nI r...",3.0,1.0,3.0,2.0,Subject: Unauthorized charges and resolution d...,2.0,2.0,2.0,3.0
6,Bank of America has frozen a XXXX transaction ...,3,2,2.0,2.0,I recently received a notification from Bank o...,3.0,1.0,2.0,2.0,"Hello Bank of America,\n\nI am writing to expr...",2.0,2.0,3.0,2.0
7,I received an email on XX/XX/XXXX from BOFA in...,3,2,NaN,NaN,Complaint:\nI recently received an email from ...,3.0,1.0,2.0,3.0,"Dear Bank of America,\n\nI am writing to expre...",2.0,2.0,2.0,3.0
8,I had fraudulent transactions on my wells farg...,3,2,3.0,2.0,"Dear Wells Fargo,\n\nI recently experienced an...",3.0,1.0,2.0,3.0,"Attention Wells Fargo,\n\nI recently experienc...",2.0,2.0,2.0,1.0
9,"I am the primary on this card, I added my boyf...",3,1,2.0,3.0,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,2.0,3.0


Export Dataset to CSV once more

In [186]:
df_test.to_csv('Mistral_deescalated200-Final.csv', index=False)